# Tokenization & Embeddings

**Converting text into numerical representations**

Here's the thing about neural networks: they only understand numbers.

Words, sentences, paragraphs? Meaningless. But vectors of floating-point numbers? Now we're talking.

So the first step in any language model is converting text into numbers the model can actually process. This happens in two stages: **tokenization** (breaking text into tokens) and **embedding** (converting those tokens into vectors).

## Our Vocabulary

For this project, we're using a tiny vocabulary with just 6 tokens:

| Token ID | Token | Description |
|----------|-------|-------------|
| 0 | `<PAD>` | Padding token (for sequences of different lengths) |
| 1 | `<BOS>` | Beginning of sequence marker |
| 2 | `<EOS>` | End of sequence marker |
| 3 | `I` | Content word |
| 4 | `like` | Content word |
| 5 | `transformers` | Content word |

In real language models, vocabularies contain thousands or tens of thousands of tokens (words, subwords, or characters). GPT-3, for instance, has 50,257 tokens. Ours has 6. This makes the calculations actually manageable while still demonstrating all the key concepts.

In [1]:
import random

# Set seed for reproducibility
random.seed(42)

# Model hyperparameters
VOCAB_SIZE = 6
D_MODEL = 16
MAX_SEQ_LEN = 5

# Token names for pretty printing
TOKEN_NAMES = ["<PAD>", "<BOS>", "<EOS>", "I", "like", "transformers"]

## Input Text

We'll train our model on a single sentence:

```
"I like transformers"
```

## Tokenization Process

To convert our text into token IDs, we:

1. Add a `<BOS>` (beginning of sequence) token at the start
2. Convert each word to its token ID
3. Add an `<EOS>` (end of sequence) token at the end

**Result:**

```
Text:      <BOS>  I     like  transformers  <EOS>
Token IDs: [1,    3,    4,    5,            2]
```

Our sequence has **length 5**.

In [2]:
# Our input sequence
tokens = [1, 3, 4, 5, 2]  # <BOS>, I, like, transformers, <EOS>
seq_len = len(tokens)

print(f"Input sequence: {tokens}")
print(f"As text: {' '.join(TOKEN_NAMES[t] for t in tokens)}")
print(f"Sequence length: {seq_len}")

Input sequence: [1, 3, 4, 5, 2]
As text: <BOS> I like transformers <EOS>
Sequence length: 5


## Language Modeling Task

In a decoder-only transformer (like GPT), the game is simple: predict the next token.

At each position, we look at all the previous tokens and try to guess what comes next. We use a **causal mask** to enforce this "no peeking at the future" rule:

| Position | Input tokens | Predict |
|----------|--------------|--------|
| 0 | `<BOS>` | `I` (token 3) |
| 1 | `<BOS> I` | `like` (token 4) |
| 2 | `<BOS> I like` | `transformers` (token 5) |
| 3 | `<BOS> I like transformers` | `<EOS>` (token 2) |

Position 4 (`<EOS>`) doesn't need to predict anything—it marks the end.

## Embeddings: Tokens to Vectors

Okay, so we have token IDs. But remember, the model needs actual vectors—continuous representations it can do math with.

We use two types of embeddings to create these vectors.

### 1. Token Embeddings

Each token gets its own embedding vector. With `d_model = 16`, that means each of our 6 tokens maps to a 16-dimensional vector.

We have a **token embedding matrix** of shape `[vocab_size, d_model]` = `[6, 16]`:

```
E_token = [
  [e₀,₀,  e₀,₁,  ..., e₀,₁₅],   ← embedding for token 0 (<PAD>)
  [e₁,₀,  e₁,₁,  ..., e₁,₁₅],   ← embedding for token 1 (<BOS>)
  [e₂,₀,  e₂,₁,  ..., e₂,₁₅],   ← embedding for token 2 (<EOS>)
  [e₃,₀,  e₃,₁,  ..., e₃,₁₅],   ← embedding for token 3 (I)
  [e₄,₀,  e₄,₁,  ..., e₄,₁₅],   ← embedding for token 4 (like)
  [e₅,₀,  e₅,₁,  ..., e₅,₁₅],   ← embedding for token 5 (transformers)
]
```

To get the embedding for token ID `i`, we simply look up row `i` of this matrix.

In [3]:
def random_vector(size, scale=0.1):
    """Generate a random vector with values ~ N(0, scale^2)"""
    return [random.gauss(0, scale) for _ in range(size)]

def format_vector(vec, decimals=4):
    """Format vector as string with specified decimal places"""
    return "[" + ", ".join([f"{v:7.{decimals}f}" for v in vec]) + "]"

# Initialize token embedding matrix
E_token = [random_vector(D_MODEL) for _ in range(VOCAB_SIZE)]

print(f"Token Embedding Matrix E_token")
print(f"Shape: [{VOCAB_SIZE}, {D_MODEL}]")
print()
for i, row in enumerate(E_token):
    print(f"  Token {i} ({TOKEN_NAMES[i]:12s}): {format_vector(row)}")

Token Embedding Matrix E_token
Shape: [6, 16]

  Token 0 (<PAD>       ): [-0.0144, -0.0173, -0.0111,  0.0702, -0.0128, -0.1497,  0.0332, -0.0267, -0.0217,  0.0116,  0.0232,  0.1164,  0.0657,  0.0111, -0.0738, -0.1015]
  Token 1 (<BOS>       ): [ 0.0246,  0.1311,  0.0042, -0.0106,  0.0532, -0.1454, -0.0312,  0.0490,  0.0873, -0.0241,  0.0377,  0.0248,  0.0782, -0.1113,  0.0568, -0.1515]
  Token 2 (<EOS>       ): [-0.2620, -0.0607, -0.0916,  0.0876,  0.0664, -0.1219,  0.0847, -0.1002, -0.0086, -0.0294,  0.0114,  0.0819,  0.0638,  0.0350,  0.0650,  0.0478]
  Token 3 (I           ): [-0.0627, -0.0717, -0.0470,  0.0499, -0.0250,  0.2336, -0.0819, -0.1099,  0.0768,  0.1422,  0.0506,  0.0836,  0.1426, -0.0094, -0.1423, -0.0532]
  Token 4 (like        ): [ 0.0953, -0.1444,  0.0034,  0.0253, -0.0316,  0.0724,  0.0581,  0.2321,  0.0620, -0.0609, -0.0562, -0.0832,  0.0952, -0.0567, -0.0070,  0.0749]
  Token 5 (transformers): [-0.0723, -0.0294, -0.1841, -0.1082, -0.0568,  0.0416,  0.1193, -0.0018,

### 2. Position Embeddings

Here's a weird thing about transformers: they have no built-in sense of order.

Unlike RNNs (which process sequences left-to-right) or CNNs (which have spatial structure), the attention mechanism treats the input like a bag of tokens. It has no idea that "I" comes before "like" unless we explicitly tell it.

So we need position embeddings. We use **learned position embeddings** (there are other approaches like sinusoidal encoding, but learned embeddings work great).

We have a **position embedding matrix** of shape `[max_seq_len, d_model]` = `[5, 16]`:

In [4]:
# Initialize position embedding matrix
E_pos = [random_vector(D_MODEL) for _ in range(MAX_SEQ_LEN)]

print(f"Position Embedding Matrix E_pos")
print(f"Shape: [{MAX_SEQ_LEN}, {D_MODEL}]")
print()
for i, row in enumerate(E_pos):
    print(f"  Position {i}: {format_vector(row)}")

Position Embedding Matrix E_pos
Shape: [5, 16]

  Position 0: [ 0.1227, -0.0030,  0.1953, -0.0359,  0.1593,  0.0115, -0.0516, -0.1128, -0.0151,  0.1423,  0.0816,  0.0689, -0.2376,  0.0711,  0.0556, -0.0550]
  Position 1: [-0.0627, -0.0002,  0.1725, -0.1055, -0.0428,  0.1362, -0.0446, -0.0364,  0.0098, -0.1241,  0.0220, -0.1210,  0.0885,  0.0003,  0.2283,  0.0281]
  Position 2: [ 0.1366, -0.1303, -0.0122,  0.0323,  0.1746, -0.1681,  0.0991,  0.0591,  0.1534,  0.0712,  0.0052, -0.0522, -0.1248,  0.0195, -0.0192,  0.2020]
  Position 3: [-0.0611,  0.0320, -0.1569, -0.0395,  0.0261,  0.0824,  0.1448, -0.0044, -0.1117,  0.0458,  0.0517,  0.0492, -0.0700,  0.1133,  0.0088,  0.0700]
  Position 4: [ 0.1274,  0.0609,  0.0287,  0.2153,  0.0244, -0.0296,  0.0112,  0.1483,  0.0119,  0.0519,  0.1196, -0.0513, -0.1727,  0.0299,  0.0230, -0.0608]


### 3. Combined Embeddings

For each token at position `i`, we add the token embedding and position embedding:

$$\text{embedding}[i] = \text{token\_embedding}[\text{token\_id}[i]] + \text{position\_embedding}[i]$$

For our sequence `[1, 3, 4, 5, 2]`:

```
Position 0: embedding[0] = E_token[1] + E_pos[0]  (for <BOS>)
Position 1: embedding[1] = E_token[3] + E_pos[1]  (for I)
Position 2: embedding[2] = E_token[4] + E_pos[2]  (for like)
Position 3: embedding[3] = E_token[5] + E_pos[3]  (for transformers)
Position 4: embedding[4] = E_token[2] + E_pos[4]  (for <EOS>)
```

In [5]:
def add_vectors(v1, v2):
    """Element-wise addition of two vectors"""
    return [a + b for a, b in zip(v1, v2)]

# Look up token embeddings for our sequence
token_embeddings = [E_token[token_id] for token_id in tokens]

# Add position embeddings to get combined embeddings
X = [add_vectors(token_embeddings[i], E_pos[i]) for i in range(seq_len)]

print("Computing combined embeddings...")
print("X[i] = E_token[token_id[i]] + E_pos[i]")
print()

for i in range(seq_len):
    token_name = TOKEN_NAMES[tokens[i]]
    print(f"Position {i} ('{token_name}'):")
    print(f"  Token embedding:    {format_vector(token_embeddings[i])}")
    print(f"  Position embedding: {format_vector(E_pos[i])}")
    print(f"  Combined X[{i}]:       {format_vector(X[i])}")
    print()

Computing combined embeddings...
X[i] = E_token[token_id[i]] + E_pos[i]

Position 0 ('<BOS>'):
  Token embedding:    [ 0.0246,  0.1311,  0.0042, -0.0106,  0.0532, -0.1454, -0.0312,  0.0490,  0.0873, -0.0241,  0.0377,  0.0248,  0.0782, -0.1113,  0.0568, -0.1515]
  Position embedding: [ 0.1227, -0.0030,  0.1953, -0.0359,  0.1593,  0.0115, -0.0516, -0.1128, -0.0151,  0.1423,  0.0816,  0.0689, -0.2376,  0.0711,  0.0556, -0.0550]
  Combined X[0]:       [ 0.1473,  0.1281,  0.1995, -0.0465,  0.2125, -0.1338, -0.0829, -0.0638,  0.0722,  0.1183,  0.1193,  0.0937, -0.1594, -0.0402,  0.1124, -0.2064]

Position 1 ('I'):
  Token embedding:    [-0.0627, -0.0717, -0.0470,  0.0499, -0.0250,  0.2336, -0.0819, -0.1099,  0.0768,  0.1422,  0.0506,  0.0836,  0.1426, -0.0094, -0.1423, -0.0532]
  Position embedding: [-0.0627, -0.0002,  0.1725, -0.1055, -0.0428,  0.1362, -0.0446, -0.0364,  0.0098, -0.1241,  0.0220, -0.1210,  0.0885,  0.0003,  0.2283,  0.0281]
  Combined X[1]:       [-0.1254, -0.0720,  0.1255,

### Final Result: Matrix X

The result is a matrix $X$ of shape `[seq_len, d_model]` = `[5, 16]`, where each row is the 16-dimensional embedding for one token in our sequence.

**This matrix X is the input to our transformer block.**

In [6]:
print("="*80)
print("FINAL COMBINED EMBEDDINGS MATRIX X")
print("="*80)
print(f"Shape: [{seq_len}, {D_MODEL}] (seq_len, d_model)")
print()
print("X =")
for i, row in enumerate(X):
    token_name = TOKEN_NAMES[tokens[i]]
    print(f"  {format_vector(row)}  # pos {i}: {token_name}")

FINAL COMBINED EMBEDDINGS MATRIX X
Shape: [5, 16] (seq_len, d_model)

X =
  [ 0.1473,  0.1281,  0.1995, -0.0465,  0.2125, -0.1338, -0.0829, -0.0638,  0.0722,  0.1183,  0.1193,  0.0937, -0.1594, -0.0402,  0.1124, -0.2064]  # pos 0: <BOS>
  [-0.1254, -0.0720,  0.1255, -0.0556, -0.0678,  0.3698, -0.1265, -0.1463,  0.0866,  0.0181,  0.0726, -0.0374,  0.2312, -0.0091,  0.0860, -0.0251]  # pos 1: I
  [ 0.2319, -0.2747, -0.0089,  0.0576,  0.1430, -0.0957,  0.1571,  0.2913,  0.2154,  0.0103, -0.0510, -0.1353, -0.0296, -0.0371, -0.0262,  0.2770]  # pos 2: like
  [-0.1334,  0.0027, -0.3410, -0.1478, -0.0307,  0.1240,  0.2642, -0.0063, -0.0856,  0.0626,  0.1602,  0.1385, -0.0427,  0.0122,  0.0991,  0.1081]  # pos 3: transformers
  [-0.1346,  0.0002, -0.0629,  0.3029,  0.0908, -0.1515,  0.0959,  0.0481,  0.0032,  0.0225,  0.1310,  0.0306, -0.1088,  0.0649,  0.0880, -0.0130]  # pos 4: <EOS>


## What's Next

These embeddings will flow through our transformer block, where the real magic happens:
1. Query, Key, Value projections (splitting into attention heads)
2. Self-attention scores (figuring out which tokens should attend to which)
3. Multi-head attention (combining information from multiple attention heads)
4. Feed-forward transformations (processing the attended representations)

Let's move on to the QKV projections.

In [7]:
# Store variables for use in subsequent notebooks
# (In a real setting, you'd save these or pass them along)
embedding_data = {
    'X': X,
    'E_token': E_token,
    'E_pos': E_pos,
    'tokens': tokens,
    'TOKEN_NAMES': TOKEN_NAMES,
    'VOCAB_SIZE': VOCAB_SIZE,
    'D_MODEL': D_MODEL,
    'MAX_SEQ_LEN': MAX_SEQ_LEN
}
print("Embedding data stored for next notebook.")

Embedding data stored for next notebook.
